### Load Poetry Dataset

In [110]:
import pandas as pd 
import random

df = pd.read_excel('../Excel_files/Full_Poem_Dataset_10-23_3.xlsx')
df = df.drop(columns=['Unnamed: 0'])
records = df.to_dict('records')
random.choice(records)

{'Text': '*\nспят спокойно мёртвые россияне ворочаются во сне\nподкладывают тёплые ладони под щёки\nутром ведут в детский сад пока что живых детей\nгде и на них найдутся\nмаленькие гимнастёрки\nигрушечные автоматы\nпесня белеет ли в поле\nмама мама что такое пороша\nмама мама что за мальчик алёша\nпочему он стоит\nзначит ли что скоро и я\nвстану над тёмным заросшим яром\nне смогу снять каменные сапоги?\nа если захочу закрыться рукой от солнца,\nсмахнуть капельку пота выступившую над переносицей\nнеужели поднять не смогу тяжелую руку?\nмама мама что делать если воздух вдруг\nразорвётся надвое, засияет?\nспят спокойно мёртвые россияне\n(прочитала в заметках к роману:\nу всех живых — прозвища, а имена у мёртвых)\nмама мамочка так страшно\nстановится анечкой и алёшей\nбольно лежать\nокровавленной\nраспростёртым',
 'Author': 'Александра Шалашова',
 'Before or after': 'After',
 'Source': 'ROAR V2',
 'Date posted': 'None'}

In [2]:
# pip install transformers

In [120]:
import torch
from transformers import BertTokenizer, BertModel

# OPTIONAL: if you want to have more information on what's happening, activate the logger as follows
import logging
#logging.basicConfig(level=logging.INFO)

import matplotlib.pyplot as plt
# % matplotlib inline

# Load pre-trained model tokenizer (vocabulary)

tokenizer = BertTokenizer.from_pretrained('DeepPavlov/rubert-base-cased')

In [90]:
text = random.choice(random.choice(records)['Text'].split('\n'))
print(text)

marked_text = "[CLS] " + text + " [SEP]"

# Tokenize our sentence with the BERT tokenizer.
tokenized_text = tokenizer.tokenize(marked_text)

# Print out the tokens.
print (tokenized_text)

не стоит человек
['[CLS]', 'не', 'стоит', 'человек', '[SEP]']


In [121]:
random.sample(list(tokenizer.vocab.keys()),10)

['Чемпионов',
 '##uga',
 'Рожер',
 'именной',
 'Milan',
 '##ящиеся',
 '##вам',
 '##ладзе',
 'смонтированы',
 'Горно']

In [129]:
# Define a new example sentence with multiple meanings of the word "bank"
text = random.choice(random.choice(records)['Text'].split('\n'))
print(text)

# Add the special tokens.
marked_text = "[CLS] " + text + " [SEP]"

# Split the sentence into tokens.
tokenized_text = tokenizer.tokenize(marked_text)

# Map the token strings to their vocabulary indeces.
indexed_tokens = tokenizer.convert_tokens_to_ids(tokenized_text)

# Display the words with their indeces.
for tup in zip(tokenized_text, indexed_tokens):
    print('{:<12} {:>6,}'.format(tup[0], tup[1]))

воды солёной вслушайся,
[CLS]           101
воды         12,240
солё         46,590
##ной         1,700
всл          72,498
##уша        29,991
##й             860
##ся          1,523
,               128
[SEP]           102


In [130]:
# Mark each of the 22 tokens as belonging to sentence "1".
segments_ids = [1] * len(tokenized_text)

print (segments_ids)

[1, 1, 1, 1, 1, 1, 1, 1, 1, 1]


In [131]:
tokenizer('следовательно')

{'input_ids': [101, 29761, 102], 'token_type_ids': [0, 0, 0], 'attention_mask': [1, 1, 1]}

In [42]:
import spacy 
nlp = spacy.load("ru_core_news_lg")

In [227]:
# Convert inputs to PyTorch tensors
data = []
for i, rec in enumerate(records):
    if i%250 == 0:
        print(f'{i}/{len(records)} recs done.')
    if not isinstance(rec['Text'],str):
        continue
    lines = rec['Text'].split('\n')
    for line in lines:
        if len(line.split(' ')) < 5:
            continue
        lemmatized_line = ' '.join([l.lemma_ for l in nlp(line)])
        doc = tokenizer.encode_plus(lemmatized_line)

        data.append({
            'doc' : doc,
            'tokens' : tokenizer.convert_ids_to_tokens(doc['input_ids']),
            'text' : line,
            'lemmatized_text' : lemmatized_line,
            'rec' : rec
        })

0/3502 recs done.
250/3502 recs done.
500/3502 recs done.
750/3502 recs done.
1000/3502 recs done.
1250/3502 recs done.
1500/3502 recs done.
1750/3502 recs done.
2000/3502 recs done.
2250/3502 recs done.
2500/3502 recs done.
2750/3502 recs done.
3000/3502 recs done.
3250/3502 recs done.
3500/3502 recs done.


In [228]:
import pickle

with open('records4RuBerta.pickle', 'wb') as handle:
    pickle.dump(data, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [230]:
# Load pre-trained model (weights)
model = BertModel.from_pretrained('DeepPavlov/rubert-base-cased',
                                  output_hidden_states = True, # Whether the model returns all hidden-states.
                                  )

# Put the model in "evaluation" mode, meaning feed-forward operation.
model.eval()


Some weights of the model checkpoint at DeepPavlov/rubert-base-cased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(119547, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (token_type_embeddings): Embedding(2, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0): BertLayer(
        (attention): BertAttention(
          (self): BertSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=False)
         

Next, let's evaluate BERT on our example text, and fetch the hidden states of the network!

*Side note: `torch.no_grad` tells PyTorch not to construct the compute graph during this forward pass (since we won't be running backprop here)--this just reduces memory consumption and speeds things up a little.*


### Sample

In [232]:
sample = [d for d in data if 'война' in d['lemmatized_text']]
len(sample)

744

In [233]:
%%time
# Run the text through BERT, and collect all of the hidden states produced from all 12 layers. 

with torch.no_grad():
    for i, d in enumerate(sample):
        if i%250 == 0:
            print(f'{i}/{len(sample)} recs done.')

        tokens_tensor = torch.tensor([d['doc']['input_ids']])
        segments_tensor = torch.tensor([d['doc']['attention_mask']])
        
        outputs = model(tokens_tensor, segments_tensor)

        # Evaluating the model will return a different number of objects based on 
        # how it's  configured in the `from_pretrained` call earlier. In this case, 
        # becase we set `output_hidden_states = True`, the third item will be the 
        # hidden states from all layers. See the documentation for more details:
        # https://huggingface.co/transformers/model_doc/bert.html#bertmodel
        hidden_states = outputs[2]
        d['hidden_states'] = hidden_states

0/744 recs done.
250/744 recs done.
500/744 recs done.
CPU times: user 1min 19s, sys: 1.96 s, total: 1min 21s
Wall time: 1min 27s


In [234]:
vecs = []
for d in sample:
    hidden_states = d['hidden_states']
    last_n_layers = 2
    # create a new dimension in the tensor
    token_embeddings = torch.stack(hidden_states, dim=0)
    # print(token_embeddings.size())

    # Remove dimension 1, the "batches"
    token_embeddings = torch.squeeze(token_embeddings, dim=1)
    # print(token_embeddings.size())

    token_embeddings = token_embeddings.permute(1,0,2)
    # print(token_embeddings.size())

    token_vecs_sum = []
    for token in token_embeddings:
        sum_vec = torch.sum(token[-2:-1], dim=0)
        token_vecs_sum.append(sum_vec)
    # print ('Shape is: %d x %d' % (len(token_vecs_sum), len(token_vecs_sum[0])))

    for i, token_str in enumerate(d['tokens']):
        # print(i, token_str)
        if token_str == 'война':
            vec = token_vecs_sum[i]
            vecs.append({
                'vec' : vec,
                'text' : d['text']
            })
print(len(vecs))

1046


In [235]:
import itertools

simtexts = []
for tup in itertools.combinations(vecs, 2):
    v1, v2 = tup
    if v1['text'] != v2['text']:
        vec1 = v1['vec']
        vec2 = v2['vec']
        sim = 1 - cosine(vec1, vec2)
        simtexts.append({
            'sim' : sim,
            'text1' : v1['text'],
            'text2' : v2['text']
        })

In [236]:
sortedtexts = sorted(simtexts, key=lambda x: x['sim'], reverse=True)

In [237]:
sortedtexts[0]

{'sim': 1,
 'text1': 'ВОЙНА ВОЙНА ВОЙНА ВОЙНА ВОЙНА ВОЙНА ВОЙНА ВОЙНА ВОЙНА ВОЙНА',
 'text2': 'ВОЙНА ВОЙНА ВОЙНА ВОЙНА ВОЙНА ВОЙНА ВОЙНА ВОЙНА ВОЙНА ВОЙНА '}

In [239]:
sortedtexts[-1]

{'sim': 0.14227454364299774,
 'text1': 'ВОЙНА ВОЙНА ВОЙНА ВОЙНА ВОЙНА ВОЙНА ВОЙНА ВОЙНА ВОЙНА ВОЙНА ',
 'text2': 'Война обнажает запах открытых ран,'}

In [247]:
for v in vecs:
    v['vec'] = v['vec'].tolist()

In [248]:
import json
with open('война-10-23.json', 'w', encoding='utf-8') as f:
    json.dump(vecs, f, ensure_ascii=False, indent=4)

### Done

In [184]:
from scipy.spatial.distance import cosine

# Calculate the cosine similarity between the word bank 
# in "bank robber" vs "river bank" (different meanings).
diff_bank = 1 - cosine(token_vecs_cat[1], token_vecs_cat[3])

# Calculate the cosine similarity between the word bank
# in "bank robber" vs "bank vault" (same meaning).
same_bank = 1 - cosine(token_vecs_cat[3], token_vecs_cat[5])

print('Vector similarity for  *similar*  meanings:  %.2f' % same_bank)
print('Vector similarity for *different* meanings:  %.2f' % diff_bank)

Vector similarity for  *similar*  meanings:  0.54
Vector similarity for *different* meanings:  0.62


In [ ]:
# print('First 5 vector values for each instance of "игра-".')
# print('')
# print("флейте   ", str(token_vecs_sum[2][:5]))
# print("футбол  ", str(token_vecs_sum[11][:5]))
# print("тубе   ", str(token_vecs_sum[20][:5]))

We can see that the values differ, but let's calculate the cosine similarity between the vectors to make a more precise comparison.

In [ ]:
print ("Number of layers:", len(hidden_states), "  (initial embeddings + 12 BERT layers)")
layer_i = 0

print ("Number of batches:", len(hidden_states[layer_i]))
batch_i = 0

print ("Number of tokens:", len(hidden_states[layer_i][batch_i]))
token_i = 0

print ("Number of hidden units:", len(hidden_states[layer_i][batch_i][token_i]))
# `hidden_states` is a Python list.
print('      Type of hidden_states: ', type(hidden_states))

# Each layer in the list is a torch tensor.
print('Tensor shape for each layer: ', hidden_states[0].size())

This looks pretty good!

## 3.5. Pooling Strategy & Layer Choice

Below are a couple additional resources for exploring this topic.

**BERT Authors**

The BERT authors tested word-embedding strategies by feeding different vector combinations as input features to a BiLSTM used on a named entity recognition task and observing the resulting F1 scores.

(Image from [Jay Allamar](http://jalammar.github.io/illustrated-bert/)'s blog)


![alt text](http://jalammar.github.io/images/bert-feature-extraction-contextualized-embeddings.png)

While concatenation of the last four layers produced the best results on this specific task, many of the other methods come in a close second and in general it is advisable to test different versions for your specific application: results may vary.

This is partially demonstrated by noting that the different layers of BERT encode very different kinds of information, so the appropriate pooling strategy will change depending on the application because different layers encode different kinds of information. 


**Han Xiao's BERT-as-service**

Han Xiao created an open-source project named [bert-as-service](https://github.com/hanxiao/bert-as-service) on GitHub which is intended to create word embeddings for your text using BERT. Han experimented with different approaches to combining these embeddings, and shared some conclusions and rationale on the [FAQ page](https://github.com/hanxiao/bert-as-service#speech_balloon-faq) of the project. 

`bert-as-service`, by default, uses the outputs from the **second-to-last layer** of the model. 

I would summarize Han's perspective by the following:

1. The embeddings start out in the first layer as having no contextual information (i.e., the meaning of the initial 'bank' embedding isn't specific to river bank or financial bank).
2. As the embeddings move deeper into the network, they pick up more and more contextual information with each layer.
3. As you approach the final layer, however, you start picking up information that is specific to BERT's pre-training tasks (the "Masked Language Model" (MLM) and "Next Sentence Prediction" (NSP)). 
    * What we want is embeddings that encode the word meaning well... 
    * BERT is motivated to do this, but it is also motivated to encode anything else that would help it determine what a missing word is (MLM), or whether the second sentence came after the first (NSP). 
4. The second-to-last layer is what Han settled on as a reasonable sweet-spot.




## 4.4. Implementations

You can use the code in this notebook as the foundation of your own application to extract BERT features from text. However, official [tensorflow](https://github.com/google-research/bert/blob/master/extract_features.py) and well-regarded [pytorch](https://github.com/huggingface/pytorch-pretrained-BERT/blob/master/examples/extract_features.py) implementations already exist that do this for you.  Additionally, [bert-as-a-service](https://github.com/hanxiao/bert-as-service) is an excellent tool designed specifically for running this task with high performance, and is the one I would recommend for production applications. The author has taken great care in the tool's implementation and provides excellent documentation (some of which was used to help create this tutorial) to help users understand the more nuanced details the user faces, like resource management and pooling strategy.

## Cite
Chris McCormick and Nick Ryan. (2019, May 14). *BERT Word Embeddings Tutorial*. Retrieved from http://www.mccormickml.com
